In [ ]:
// Importieren der benötigten Module
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE, 0.0.2"
#r "nuget: FSharp.Stats"

// ... und öffnen der benöätigten Module
open FsODE
open Plotly.NET
open FSharp.Stats
open FsODE.Stochastic

// der Rechenalgorithmus der Programmbibliothek, die wir benutzen:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

Installed Packages FSharp.Stats, 0.4.8 FsODE, 0.0.2 Plotly.NET.Interactive, 3.0.2

## Modellierung von einfachen biologischen Systemen

### Inhalt

- einfache Biologische Systeme
- Stochastik 
- Aktivator und Repressor
- Autoregulation
    - Negative Autoregulation
    - Postive Autoregulation

### Biologische Systeme 

Biologische Systeme können als Netzwerke aufgefasst werden, in welchen biologischen Einheiten miteinander verknüpft sind. Dabei hängen Größe, Art, und Anzahl der biologischen Einheiten vom gewählten System ab. Biologische Zellen sind dabei besonders interessanteste Systeme. Sie werden begrenzt von einer Membran die Stoff-, Energie-, und Informationsaustausch mit der Umgebung erlaubt, und enthalten Millionen von Molekülen die alle miteinander interagieren. Dabei sind die Interaktionen zumeist spezifischer Natur und formen selbst wiederum kleinere biologische Systeme, die die Funktion der ganzen Zelle sicherstellen. Beispiele für solche Systeme sind die Kontrolle des Zelltods, Signalwege die die Kommunikation mit anderen Zellen erlauben, die Nutzung von Metaboliten, und viele weitere. 
Die Grundeinheiten in solchen Systemen sind Gene, beziehungsweise die Proteine, welche von diesen kodiert werden. Diese Grundeinheiten sind miteinander verbunden in sogenannten Netzwerkmotiven. Diese Netzwerkmotive sind mannigfaltig und die Grundlegendsten werden in den nächsten zwei Notebooks besprochen. <br>
Alle Netzwerke beschäftigen sich damit, wie sich die einzelnen Proteinkonzentration verändern und welche Auswirkungen dies hat. Daher ist der erste Schritt zum Verständnis von diesen, die Differenzialgleichung die beschreibt wie sich die Proteinkonzentration eines Proteins über die Zeit verändert.
Der Aufbau der Differenzialgleichung ist immer identisch. Die Komplexität der einzelnen Teile der Gleichung kann jedoch variieren in Abhängigkeit der Anzahl an Faktoren, die auf die Produktion des Proteins Einfluss nehmen. Diese allgemeine Form ist 
$$ \frac {dProteinkonzentration} {dt} = \textrm{Produktion des Proteins} - \textrm{Verringerung der Proteinkonzentration}$$


##### Aufgabe 3.1
***Der Ausdruck "Verringerung der Proteinkonzentration" ist sehr allgemein formuliert. Überlegen Sie, welche Faktoren für diese Verringerung verantwortlich sein können. Betreiben Sie falls nötig Literaturrecherche.***


Was ist dann die Proteinkonzentration eines Proteins X das von einem Gen x produziert wird? Wenn die Annahme getätigt wird, dass das Gen x nur das Protein X codiert und auch in sonst keiner Weise beeinflusst wird, kann die Produktion des Proteins X direkt mit einer Produktionsrate $\beta$ beschrieben werden. Dem gegenüber stehen die Faktoren, welche die Proteinkonzentration verringern. Diese sind abhängig von der Proteinkonzentration. Die mathematische Beschreibung dieses Prozesses wird mit dem Verringerungsfaktor $\gamma$ zusammengefasst, der mit der Proteinkonzentration verrechnet wird. Die Differenzialgleichung ist somit

$$ \frac {dProteinkonzentration} {dt} = \beta - \gamma*Proteinkonzentration$$

##### Aufgabe 3.2
***Setze die Differenzialgleichung in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole).***


In [ ]:
//Konstanten
let beta =    2. // Konstante für die Zunahme der Proteinkonzentration
let gamma =   1. // Konstante für die Verringerung der Proteinkonzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonzentration = P[0] 
        // Modelfunktion (hier die Gleichung einfügen)
        let Proteinkonzentration' = beta - gamma*Proteinkonzentration

        [| Proteinkonzentration' |]

//Startkonzentrationen
let P0 = [| 
    10.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

let proteins = SolPoints.toPoints 1 Ps

//Graphische Darstellung der numerischen Lösung
proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

Der erstellte Graph zeigt wie sich die Proteinkonzentration über die Zeit verändert. Die Proteinkonzentration startet am Zeitpunkt 0 bei einem Wert von 10, $Proteinkonzentration (0) = 10$. Die Proteinkonzentration nähert sich mit der Zeit dem Wert 2. Dieser Wert entspricht dem Punkt an dem Produktion und Verringerung der Proteinanzahl im Gleichgewicht stehen und ist somit ein Gleichgewichtspunkt. Dieser Gleichgewichtspunkt wird im folgenden $Proteinkonzentration_{GGW}$ genannt. Das $Proteinkonzentration_{GGW}$ ist von den Werten von $\beta$ und $\gamma$ anhängig. Die Werte von $\beta$ und $\gamma$ die zu dem $Proteinkonzentration_{GGW} = 2$ geführt haben sind 2. und 1. (die Werte sind im oberen code block beim Absatz Konstanten zu finden).

##### Aufgabe 3.3 
***Verändere den Wert von $\beta$ und versuche eine Gesetzmäßigkeit zu erkennen die beschreibt welchen Wert die Proteinkonzentration im GGW annimmt in Abhängigkeit von $\beta \textrm{ und } \gamma$. Wähle die korrekte Gleichung aus den vier nachfolgen aus, die diese Gesetzmäßigkeit beschreibt. Kopiere dafür den Code Block von oben in den nachfolgenden Code Block und ändere dann $\beta$ ( $\beta$ befindet sich im Abschnitt Konstanten, beachte das die Eingesetzten Zahlen Fließzahlen sein müssen).***

In [ ]:
//Code hier reinkopieren

Mögliche Gleichungen die das Problem beschreiben:


a) $Proteinkonzentration_{GGW} = \gamma  - 2*\beta$
 
b) $Proteinkonzentration_{GGW} = \frac {\beta * \gamma}  {\gamma^{2}}$ 

c) $Proteinkonzentration_{GGW} = \frac {\beta}  {\gamma}$ 

d) $Proteinkonzentration_{GGW} = \frac {\beta^{2}}  {\gamma^{\beta}}$ 

In diesen Block die gewählte Antwort einfügen

Natürlich werden Proteine nicht direkt von einem Gen produziert, sondern es kommt zuerst zur Transkription des Gens und somit zur Produktion von mRNA. Daher ist eine korrekter Darstellung der Proteinkonzentration nur möglich durch das verknüpfen von zwei Differenzialgleichungen. Eine die beschreibt wie sich die mRNA Konzentration mit der Zeit ändert und eine die beschreibt wie sich die Proteinkonzentration mit der Zeit ändert. Diese sind 
$$\textrm {mRNAKonzentration'}  = \frac {\textrm {dmRNAKonzentration}} {dt} = \beta_{1} - \gamma_{1}*\textrm {mRNAKonzentration'}$$
$$\textrm {Proteinkonzentration'} = \frac {dProteinkonzentration} {dt} = \beta_{2} * mRNAKonzentration - \gamma_{2}*Proteinkonzentration$$

Die verknüpfung findet bei der Produktion des Proteins statt. Diese ist jetzt abhängig von der Konzentration von mRNA in der Zelle und nicht länger nur von der Konstante $\beta$, was auch nur logisch ist da mehr mRNA zu mehr Protein führen sollte. 

##### Aufgabe 3.4
***Setze die beiden Differenzialgleichung in das folgende Model ein (beachte das $\beta_{1} \textrm{ und } \gamma_{1}$ als "beta1" und "gamma1", ohne "", eingesetzt werden müssen und nicht als Symbole. Das selbe gilt für $\beta_{2} \textrm{ und } \gamma_{2}$ ).***

In [ ]:
//Konstanten
let beta1 = 1.  // Konstante für die Zunahme der Konzentration an mRNA
let gamma1 = 2. // Konstante für die Verringerung der Konzentration an mRNA
let beta2 = 2.  // Konstante für die Zunahme der Proteinkonzentration
let gamma2 = 1. // Konstante für die Verringerung der Proteinkonzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let mRNAKonzentration = P[0] 
        let Proteinkonzentration = P[1]
        // Modelfunktion (hier die Gleichung einfügen)
        let mRNAKonzentration' = beta1 - gamma1 * mRNAKonzentration
        let Proteinkonzentration' = beta2 * mRNAKonzentration - gamma2 * Proteinkonzentration

        [| mRNAKonzentration'; Proteinkonzentration' |]

//Startkonzentrationen
let P0 = [|
    0.0 
    0.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

let mRNA = SolPoints.toPoints 1 Ps
let proteins = SolPoints.toPoints 2 Ps

//Graphische Darstellung der numerischen Lösung
[
    let mRNA = SolPoints.toPoints 1 Ps
    mRNA
    |> Chart.Spline 
    |> Chart.withTraceInfo("mRNA")
    let proteine = SolPoints.toPoints 2 Ps
    proteine
    |> Chart.Spline 
    |> Chart.withTraceInfo("Proteine")
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Konzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.5
***Wie hängt der Gleichgewichtszustand der Konzentration von mRNA mit dem Gleichgewichtszustand vom Protein zusammen? 
Stellen sie die mathematische Gleichung für den Gleichgewichtszustand vom Protein auf.
Falls sie Probleme mit dem finden der Gleichung haben, dann kopieren sie den Code Block von oben in den folgenden Code Block und verändern sie die Werte von $beta_{1}$ und $\gamma_{1}$, um zu sehen wie sich die beiden Gleichgewichtszustände bei verschiedenen Werten ändern.***

In [ ]:
//Hier Code einfügen falls nötig

Hier die Antwort eingeben.

Es folgt eine kurze Besprechung. Falls sie zu schnell vorangekommen sind, können sie die nachfolgende Frage bearbeiten.<br>

##### Expertenaufgabe 3.1
***Ein Protein wird mit einer konstanten rate $\beta_{1}$ produziert. Die Produktionsrate ändert sich plötzlich zur Produktionsrate $\beta_{2}$. Stelle die Differenzialgleichung für die Proteinkonzentration des Proteins nach der Zeit auf und füge diese in den unten stehenden Code Block ein. <br>
Erkläre den Lösungsweg und welche Komponenten im Code Block geändert wurden. Bitte ändere nur die Abschnitte bei denen in den Kommentaren steht "kann verändert werden". Tipp: Nicht alle Stellen wo steht "kann verändert werden" müssen auch verändert werden.***

In [ ]:
//Konstanten (kann verändert werden)
let beta = 0.1  // Konstante für die Zunahme der Proteinkonzentration
let gamma = 0.1 // Konstante für die Verringerung der Proteinkonzentration

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonzentration = P[0] 
        // Modelfunktion (kann verändert werden)
        let Proteinkonzentration' = beta - gamma * Proteinkonzentration

        [| Proteinkonzentration' |]

//Startkonzentrationen (kann verändert werden)
let P0 = [| 
    0.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

let proteins = SolPoints.toPoints 1 Ps

//Graphische Darstellung der numerischen Lösung
proteins
|> Chart.Spline 
|> Chart.withTraceInfo("Protein")
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

-------------

## Stichproben aus Verteilungen

Für einige Situationen kennen wir die Wahrscheinlichkeitsverteilung. Wenn die Situation zum Beispiel so aussieht, dass wir n
unabhängige Münzwürfe machen, jeder mit der Wahrscheinlichkeit p, dass Kopf fällt, dann ist die Anzahl h
der Köpfe binomialverteilt. Wir könnten beweisen, dass dies der Fall ist, oder es in einem Buch nachschlagen. Aber manchmal ist es zu schwierig (oder unmöglich) zu beweisen, wie die Verteilung aussieht. Daher können wir ihre Eigenschaften numerisch berechnen, indem wir eine Stichprobe aus der Verteilung nehmen. Für die Stichprobe wird ein Zufallszahlengenerator verwendet, um die Situation zu simulieren, aus der die Verteilung resultiert. Wenn Sie also die Situation kennen und einen Computer zur Hand haben, können Sie über eine Stichprobe ein Diagramm der Verteilung erhalten.

Lassen Sie uns dies anhand der Binomialverteilung demonstrieren. Wir nehmen n=25
und p=0.25 und berechnen P(h∣n,p), die Wahrscheinlichkeit, dass h Köpfe bei n Münzwürfen mit einer Wahrscheinlichkeit p für Köpfe auftreten. Wir ziehen 10, 30, 100 und 300 Stichproben und stellen sie der erwarteten Binomialverteilung gegenüber.

In [ ]:
let stichproben = [|30;100;1000;10000|]
let n = 25
let p = 0.25

let binomialverteilung = Distributions.Discrete.Binomial.Init p n

let binomialPunkte =
    [|0 .. 25|]
    |> Array.map (fun x -> x,binomialverteilung.PMF x)

stichproben
|> Array.map (fun s ->
    [
        simulateCoinflips n p s
        |> Array.countBy id
        |> Array.map (fun (heads,count) -> heads, float count/ float s)
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Orange")
        binomialPunkte
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Blue", Opacity = 0.3)
    ]
    |> Chart.combine
    |> Chart.withLegend false
    |> Chart.withXAxisStyle(TitleText = $"h<br>{s} Stichproben")
    |> Chart.withYAxisStyle(TitleText = "P(h)")
)
|> Chart.Grid(2,2)
|> Chart.withSize (800,800)

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.6

***Die meisten Münzen haben eine Wahrscheinlichkeit von p=0.5 für Kopf. Simulieren Sie diese Situation wie wie oben gezeigt und vergleichen Sie die Verteilungen aus den unterschiedlichen Stichprobenzahlen mit der tatsächlichen Binomialverteilung.***

In [ ]:
// Kopieren sie den benötigten Code aus dem vorherigen Codeblock
let stichproben = [|30;100;1000;10000|]
let n = 25
let p = 0.5

let binomialverteilung = Distributions.Discrete.Binomial.Init 0.5 25
let binomialPunkte =
    [|0 .. 25|]
    |> Array.map (fun x -> x,binomialverteilung.PMF x)

stichproben
|> Array.map (fun s ->
    [
        simulateCoinflips n p s
        |> Array.countBy id
        |> Array.map (fun (heads,count) -> heads, float count/ float s)
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Orange")
        binomialPunkte
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromString "Blue", Opacity = 0.3)
    ]
    |> Chart.combine
    |> Chart.withLegend false
    |> Chart.withXAxisStyle(TitleText = $"h<br>{s} Stichproben")
    |> Chart.withYAxisStyle(TitleText = "P(h)")
)
|> Chart.Grid(2,2)
|> Chart.withSize (800,800)

Wie wir sehen, können wir die tatsächliche Verteilung annähernd berechnen, wenn wir eine Stichprobe aus der Wahrscheinlichkeitsverteilung ziehen. Je mehr Stichproben wir nehmen, desto besser wird die Annäherung.
Stichproben sind eine so leistungsfähige Strategie, dass hocheffiziente Algorithmen mit praktischen APIs entwickelt wurden, um Stichproben aus benannten Wahrscheinlichkeitsverteilungen zu ziehen. Wir hätten zum Beispiel `binomialverteilung.Sample()` als einfachen (und viel effizienteren) Ersatz für die obige Funktion `simulateCoinflips` verwenden können.

Wir werden die gleiche Strategie für die Lösung von stochastischen Gleichungssystemen verwenden. Wir werden eine stochastische Methode verwenden, um eine Stichprobe aus der Verteilung zu ziehen, die von dem Gleichungssystem beschrieben wird. Diese Methode wurde in den letzten 70er Jahren von Dan Gillespie entwickelt. Aus diesem Grund werden diese Stichprobenverfahren oft als Gillespie-Simulationen bezeichnet. Der Algorithmus wird manchmal auch als stochastischer Simulationsalgorithmus oder SSA bezeichnet.

Im Folgenden werden wir die Funktionsweise dieses Algorithmus anhand der einfachen Produktion eines Proteins untersuchen.

## Simulieren des stochastischen Gleichungssystems

Für die einfache Proteinproduktion gibt es die folgende Reaktion:

DNA→mRNA→Protein

Wir haben bereits vorher die Proteinproduktion mit Hilfe von Differentialgleichungen simuliert. Das Ergebnis davon ist jedoch nicht für jeden Fall anwendbar. Bei dieser Simulation haben wir den Mittelwert der Produktion bei einer großen Zahl an Reaktionen betrachtet. Sobald aber stochastische Effekte eine größere Rolle spielen (z.B. das Gesetz der großen Zahlen nicht anwendbar ist und mit wenig Reaktionen gearbeitet wird), sind stochastische Simulationen eine geeignetere Methode.

## Anwenden der Gillespie Simulation

Um die Gillespie-Simulation zu verwenden, erstellen wir zunächst ein Array, das die Änderungen in den Zahlen von mRNA (m)
und Protein (p) für jede der vier möglichen Reaktionen enthält. Auf diese Art und Weise werden die Aktualisierungen der Teilchenzahlen kodiert, die wir durch die Wahl der jeweiligen Zustandsänderungen erhalten.

In [ ]:
// Index 0 repräsentiert Änderung bei mRNA (m), Index 1 repräsentiert Änderung bei Protein (p)
let simpleUpdate =
    [|
        1., 0.;  // mRNA Produktion
        -1., 0.; // mRNA Abbau
        0., 1.;  // Protein Produktion
        0., -1.  // Protein Abbau
    |]

Zusätzlich benötigen wir die Übergangswahrscheinlichkeiten der Zustandsänderungen. Die Übergangswahrscheinlichkeiten werden im Rahmen der stochastischen Simulation auch als Propensities bezeichnet.
Dafür erstellen wir eine Funktion, die das Array der Propensities für jede der vier Reaktionen aktualisiert. Wir aktualisieren die Propensities (die als Argument an die Funktion übergeben werden), anstatt sie zu instanziieren und zurückzugeben, um bei der Ausführung des Codes Speicherplatz zu sparen. Sie wird eine Funktion der aktuellen Anzahl von Molekülen sein. Im Allgemeinen kann sie auch eine Funktion der Zeit sein, daher berücksichtigen wir ausdrücklich auch die Zeitabhängigkeit (auch wenn wir sie in diesem einfachen Beispiel nicht verwenden).

In [ ]:
//Updates einen Array von Propensities abhängig von den Parametern und der Population
let simplePropensities (propensities: float []) population t beta_m beta_p gamma =
    let m,p = population
    propensities.[0] <- beta_m     // mRNA Produktion
    propensities.[1] <- m          // mRNA Abbau
    propensities.[2] <- beta_p * m // Protein Produktion
    propensities.[3] <- gamma * p  // Protein Abbau

## "Ziehen" einer Reaktion

Anschließend brauchen wir eine allgemeine Funktion, die eine bestimmte Reaktion und das Zeitintervall für diese Reaktion zieht. Um das Zeitintervall zu bestimmen, ziehen wir zunächst eine Zufallszahl aus einer Exponentialverteilung.
Als nächstes müssen wir auswählen, welche Reaktion stattfinden soll. Das läuft darauf hinaus, eine Stichprobe aus einer diskreten Verteilung zu ziehen.
Dabei ist die Wahrscheinlichkeit jeder Reaktion proportional zu ihrer Propensity.

Hier können sie die dafür verwendete Funktion ausprobieren. Die Funktion gibt mit der an dem jeweiligen Index angegebenen Wahrscheinlichkeit den Index bei Aufruf der Funktion zurück.

In [ ]:
// Beispielspropensitäten
let probs = [|
        0.1    // Wahrscheinlichkeit Index 0 zu ziehen
        0.3    // Wahrscheinlichkeit Index 1 zu ziehen
        0.4    // Wahrscheinlichkeit Index 2 zu ziehen
        0.05   // Wahrscheinlichkeit Index 3 zu ziehen
        0.15   // Wahrscheinlichkeit Index 4 zu ziehen
    |]
sampleDiscrete probs

4

##### Aufgabe 3.7

***Führen sie die Funktion `sampleDiscrete` mit Werten Ihrer Wahl mehrfach aus. Stimmen die Ziehungen der Indices mit dem Verhältnis der Zahlen überein?***

In [ ]:
// Hier könnte Ihr Code stehen

# Besprechung

## SSA Zeitsprungverfahren

Jetzt können wir unsere SSA-Hauptschleife verwenden. Wir werden nur die Zahlen zu vorher festgelegten Zeitpunkten speichern. Das spart Arbeitsspeicher, und wir interessieren uns ohnehin nur für die Werte zu bestimmten Zeitpunkten.

Beachten Sie, dass diese Funktion generisch ist. Alles, was wir brauchen, um unser System zu spezifizieren, ist das Folgende:

    - Eine Funktion zum Berechnen der Übergangswahrscheinlichkeiten
    - Wie die Aktualisierungen für eine bestimmte Reaktion vorgenommen werden
    - Ausgangspopulation

Zusätzlich geben wir die notwendigen Parameter, eine Anfangsbedingung und die Zeitpunkte an, zu denen wir unsere Stichproben speichern wollen.

Wir können nun eine Reihe von SSA-Simulationen durchführen und die Ergebnisse speichern. Wir führen dafür 100 Trajektorien mit den Werten $\beta_{p}$ = 10, $\beta_{m}$ = 10 und $\gamma$ = 0,4 durch.

In [ ]:
let zeitpunkte = [|0. .. 0.5 .. 50.|]

let samples =
    [|
        // So oft wird die Reaktion simuliert (100 mal)
        for i = 0 to 100 do
            gillespieSSA simplePropensities simpleUpdate (0,0) zeitpunkte 10. 10. 0.4
    |]

In [ ]:
[
    samples
    |> Array.map fst
    |> Array.transpose
    |> Array.map (fun numbers -> Array.average numbers, Seq.stDev numbers)
    |> Array.unzip
    |> fun (number,stDev) ->
        Chart.Line (zeitpunkte, number, LineWidth = 5)
        |> Chart.withYErrorStyle(Array = stDev, Color = Color.fromString "grey")
        |> Chart.withYAxisStyle(TitleText = "Anzahl")
        |> Chart.withXAxisStyle(TitleText = "Zeit")
        |> Chart.withTraceInfo("mRNA")
    samples
    |> Array.map snd
    |> Array.transpose
    |> Array.map (fun numbers -> Array.average numbers, Seq.stDev numbers)
    |> Array.unzip
    |> fun (number,stDev) ->
        Chart.Line (zeitpunkte, number, LineWidth = 5)
        |> Chart.withYErrorStyle(Array = stDev, Color = Color.fromString "grey")
        |> Chart.withYAxisStyle(TitleText = "Anzahl")
        |> Chart.withXAxisStyle(TitleText = "Zeit")
        |> Chart.withTraceInfo("Protein")
]
|> Chart.Grid (1,2)
|> Chart.withSize(1200., 600.)

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.8
***Führen sie `gillespieSSA` mit den obigen Werten unterschiedlich oft aus. Wie sehen die Kurven bei niedriger Wiederholungszahl aus? Wie bei hoher?***

In [ ]:
// Kopieren sie den benötigten Code aus dem vorherigen Codeblock

Abschließend können wir die Wahrscheinlichkeitsverteilungen im stationären Zustand anschauen. Dafür schauen wir uns die Häufigkeiten der einzelnen Werte als normiertes Histogramm mit Bingröße=1 an. Dies entspricht der Wahrscheinlichkeitsdichtefunktion (PDF).

In [ ]:
let samplesmRNA =
    samples
    |> Array.map fst
    |> Array.collect (fun x -> x.[50 ..])

let sampleZahlenmRNA =
    samplesmRNA
    |> Array.countBy id
    |> fun histo ->
        let histoSum =
            histo
            |> Array.map snd
            |> Array.sum
        histo
        |> Array.map (fun (event,count) -> event, float count/ float histoSum)

let histomRNA =
    Chart.Column(sampleZahlenmRNA)
    |> Chart.withXAxisStyle(TitleText = "mRNA Anzahl")
    |> Chart.withYAxisStyle(TitleText = "P(Anzahl)")

histomRNA

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let samplesProtein =
    samples
    |> Array.map snd
    |> Array.collect (fun x -> x.[50 ..])

let sampleZahlenProtein =
    samplesProtein
    |> Array.countBy id
    |> fun histo ->
        let histoSum =
            histo
            |> Array.map snd
            |> Array.sum
        histo
        |> Array.map (fun (event,count) -> event, float count/ float histoSum)

let histoProtein =
    Chart.Column(sampleZahlenProtein)
    |> Chart.withXAxisStyle(TitleText = "Protein Anzahl")
    |> Chart.withYAxisStyle(TitleText = "P(Anzahl)")

histoProtein

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.9
***Welcher Verteilung entspricht die mRNA-, und welcher die Protein-Verteilung? Probieren sie die Verteilungen aus und bestimmen sie die passenden Parameter.***

In [ ]:
let poissonLambda = Distributions.Discrete.Poisson.Fit samplesmRNA
let poissonVerteilung = Distributions.Discrete.Poisson.Init poissonLambda

let gammaAlpha,gammaBeta = Distributions.Continuous.Gamma.Fit samplesmRNA
let gammaVerteilung = Distributions.Continuous.Gamma.Init gammaAlpha gammaBeta

// let betaAlpha,betaBeta = Distributions.Continuous.Beta.Fit samplesmRNA
// let betaVerteilung = Distributions.Continuous.Beta.Init betaAlpha betaBeta

[
    histomRNA
    Chart.Line ([|0. .. 25.|], [|0 .. 25|] |> Array.map poissonVerteilung.PMF)
    Chart.Line ([|0. .. 25.|], [|0 .. 25|] |> Array.map gammaVerteilung.PDF)
    // Chart.Line ([|0. .. 25.|], [|0 .. 25|] |> Array.map betaVerteilung.PDF)
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let gammaAlpha,gammaBeta = Distributions.Continuous.Gamma.Fit samplesProtein
let gammaVerteilung = Distributions.Continuous.Gamma.Init gammaAlpha gammaBeta

[
    histoProtein
    Chart.Line ([|0. .. 500.|], [|0. .. 500.|] |> Array.map gammaVerteilung.PDF)
]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

# Besprechung

### Regulation

#### Repressoren

Wie bereits in der Einführung erwähnt ist wichtig, das mathematische Modell dem zu untersuchenden System anzupassen. In echten biologischen Systemen werden Gene nicht konstant exprimiert, sondern unterliegen einer Regulation, die von unterschiedlichen Faktoren abhängt. Die einfachste Form der Regulierung ist mittels Repressoren oder Aktivatoren. Beides sind zumeist Proteine, die an spezifische Regionen auf der DNA binden können um die Promotoraktivität eines Gens zu regulieren, wobei die Stärke der Regulierung meist mit der Stärke der Bindung des Regulators zur DNA-Region einhergeht. <br>
![Promotor](https://www.promotor.de/wp-content/uploads/2022/04/Gebrauchtwagenmarkt-Pressemitteilung.jpg)<br>
Um den Einfluss eines Repressors/Aktivators auf die Produktion von einem Protein zu bestimmen, ist es zuerst nötig sich mit dem Prozess der Bindung des Repressors/Aktivators an die jeweilige Bindestelle, durch die die Proteinprodukion beeinflusst wird, zu beschäftigen. Innerhalb der Zelle bindet und löst sich ein Repressor von seiner Bindestelle in einem kontinuierlichen Prozess. Dies kann als eine Gleichgewichtsreaktion dargestellt werden
$$D + R\quad \underset{k_{-}}{\stackrel{k_{+}}{\rightleftharpoons}} \quad D_{gebunden}$$
wobei $D$ die Konzentration von Bindestellen ohne Repressor,<br> $R$ die Repressorkonzentration,<br> $D_{gebunden}$ die Konzentration von Bindestellen mit Repressor,<br> $k_{+}$ die Geschwindigkeit des Bindens des Repressor an die Bindestelle,<br> und $k_{-}$ die Geschwindigkeit des Loslösens des Repressors von der Bindestelle ist.<br> $D$ und $D_{gebunden}$ zusammen ergeben $D_{total}$ was der Konzentration aller Bindestellen entspricht ($D_{total} = D + D_{gebunden} $).<br>
Das Gen wird nur dann stark exprimiert wenn der Repressor nicht gebunden ist, was dem Verhältnis von nicht gebundenen Bindestellen zu der Anzahl von allen Bindestellen entspricht, also $D/D_{total}$. Für eine Differenzialgleichung die beschreibt wie sich die Proteinkonzentration verhält wenn das Gen von einem Repressor reguliert wird ist es daher nötig die Produktionsrate mit dem Verhältnis $D/D_{total}$ zu multiplizieren. Sowohl $D$ als auch $D_{total}$ sind aber nur schwierig experimentell zu bestimmen. Die Repressorkonzentration und die Dissoziationskonstante, also die Konzentration  bei der die Produktion um die Hälfte reduziert ist, sind hingegen vergleichsweise einfach zu bestimmen. Daher ist es sinnvoll das Verhältnis $D/D_{total}$ als einen Zusammenhang von $R$ und der Dissoziationskonstante $K_{D}$ darzustellen. Dies ist möglich durch das Umformen der Gleichgewichtsgleichung die sich aus der Gleichgewichtsreaktion ergibt. Diese ist:
$$k_{+}*D*R = k_{-}*D_{gebunden}$$
Einsetzten von $D_{total} = D + D_{gebunden} $ um $D_ {gebunden}$ zu ersetzen ergibt:
$$k_{+}*D*R = k_{-}*(D_{total}-D)$$
Separieren von $D$ und $D_{total}$ von den anderen Faktoren führt zu:
$$\frac {D} {(D_{total}-D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D/D} {(D_{total}/D-D/D)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {1} {(D_{total}/D-1)}= \frac {k_{-}}{k_{+}*R} $$
$$\frac {D_{total}}{D}-1= \frac {k_{+}*R}{k_{-}} $$
$$\frac {D_{total}}{D}= \frac {k_{+}*R}{k_{-}}+1 $$
das Verhältnis von $\frac {k_{-}}{k_{+}}$ kann ersetzt werden durch die Dissoziationskonstante $K_{D}$. Daher folgt:
$$\frac {D_{total}}{D}= \frac {R}{K_{D}} +1 $$
$$\frac {D}{D_{total}}= \frac {1}{1+\frac {R}{K_{D}}} $$

Der Term $\frac {1}{1+\frac {R}{K_{D}}}$ beschreibt somit den Term durch den die Expressionsrate eines Gens gedämpft wird, wenn es von einem Repressor reguliert wird. Das heißt wenn das Gen x, welches das Protein X produziert, durch den Inhibitor R reguliert wird, kann dies mit folgender Differenzialgleichung beschrieben werden
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {R}{K_{D}}}  - \gamma*Proteinkonzentration$$





##### Aufgabe 3.10
***Setze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor reguliert wird in das folgende Model ein (beachte das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***


In [ ]:
//Konstanten
let alpha =    0.5  // Konstante für die Zunahme der Proteinkonzentration (nicht reguliert)
let beta =    50.0  // Konstante für die Zunahme der Proteinkonzentration (reguliert)
let gamma =   20.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =     20.0  // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.0  // Repressorkonzentration


// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let x = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonzentration = P[1] // Änderung in der Proteinmenge (reguliert durch Repressor)
        //Änderung der Proteinkonzentration ohne einen Repressor (nicht verändern bitte)
        let x' = beta - (gamma * x)
        // Änderung des Proteinkonzentration unter Einfluss von einem Repressor (hier die Gleichung einsetzen)
        let Proteinkonzentration' = beta * 1. / (1. + (R / K_d)) - (gamma * Proteinkonzentration)

        [| x'; Proteinkonzentration' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0  
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        1., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzhal der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins
    |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = SolPoints.toPoints 2 Ps
    proteinsregulated
    |> Chart.Spline 
    |> Chart.withTraceInfo("Reguliert");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.11
***Erklären Sie, wie sich der Repressor auf die Höhe des Gleichgewichtszustandes der Proteinkonzentration, $Proteinkonzentration_{GGW}$, und auf die Zeit die benötigt wird diesen zu erreichen auswirkt. Verändern sie dafür die Werte von "R" falls nötig. Beachten sie das die eingesetzten Werte reale Zahlen sein müssen und nicht ganze Zahlen. Falls sie die Werte verändern möchten, dann kopieren sie den oben stehnden Code Block und setzten sie diesen in den nachfolgenden Code Block ein. Verändern sie dann den Wert von R (R befindet sich im Bereich Konstanten).***


In [ ]:
//Code hierrein kopieren

In diesen Block die Antwort schreiben

##### Aufgabe 3.12
***In der Realität wird ein Gen schwach exprimiert, auch wenn die Expression des Gens durch einen Repressor gehemmt wird. Ergänze die Differenzialgleichungen für ein Protein dessen Produktion von einem Repressor inhibiert wird um einen Term $\alpha$ der diese Grundgenexpression wiederspielt. Beachte dabei das $\alpha$ nicht vom Repressor beeinflusst wird (setze $\alpha$ als "alpha", ohne "", in die Gleichung ein).<br>
Kopiere dafür den Code Block von oben in den nachfolgenden Code Block und ändere die Differenzialgleichung in diesem.***



In [ ]:
//Code hierrein kopieren
//Anschliesend die Differenzialgleichung um alpha erweitern. Bitte beachten das sie alpha auch zu der Liste der Konstanten hinzufügen müssen
//und dementsprechend alpha auch einen Wert zuweisen müssen.

***Wie wirkt sich $\alpha$ auf den Gleichgewichtszustand aus?***

In diesen Block die Antwort schreiben

Es folgt eine kurze Besprechung. 

-----------

#### Aktivatoren

Hier Formel

##### Aufgabe 3.13
***Stelle die Differenzialgleichung für die Proteinproduktion auf, die von einem Aktivator reguliert wird. Beachte dabei das ein Aktivator gegensätzlich zu einem Repressor wirkt und daher der Faktor mit dem $beta$ multipliziert werden muss $\frac {D_{gebunden}}{D_{total}}$ ist. Dies bedeutet das sie folgende Gleichung in diese Form umwandeln müssen***
$$k_{+}*(D_{gebunden}-D_{total})*R = k_{-}*D_{gebunden}$$
***und anschließend in die Differenzialgleichung für die Proteinproduktion einsetzen müssen***
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {D_{gebunden}}{D_{total}}  - \gamma*Proteinkonzentration$$

***Falls sie Probleme mit dem Umformen haben, schauen sie sich noch einmal die Umformung für den Repressor an.***

//////

***Wie unterscheiden sich die Formeln, die die Genexpression unter Repressor- und Aktivatoreinfluss beschreiben? Versuchen Sie, diesen Unterschied biologisch zu begründen.***




In diesen Block die Antwort schreiben

##### Aufgabe 3.14

***Setze die Differenzialgleichungen für ein Protein das von einem Aktivator und die Differenzialgleichungen für ein Protein das von einem Repressor reguliert wird in das folgende Model ein (beachten sie dabei, das sie die Proteinkonzentration für die Gleichung die von einem Repressor beeinflusst wird "Proteinkonzentration2, ohne "", nennen und die Proteinkonzentration für die Gleichung die von einem Aktivator beeinflusst wird "Proteinkonzentration3, ohne "", nennen. Bechte ebenfalls das  $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird)***

In [ ]:
//Konstanten
let beta =    10.  // Konstante für die Zunahme der Proteinkonzentration
let gamma =   5.   // Konstante für die Verringerung der Proteinkonzentration
let K_d =     2.   // Repressorkonzentration bei der die Proteinproduktion um die Hälfte reduziert ist
let R =       10.  // Repressorkonzentration
let A =       10.  // Aktivatorexpression 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge (reguliert durch Repressor)
        let Proteinkonzentration3 = P[2] // Änderung in der Proteinmenge (reguliert durch Aktivator)
        //Änderung der Proteinkonzentration ohne einen Repressor
        let Proteinkonzentration1' = beta - (gamma*Proteinkonzentration1)
        // Änderung des Proteinkonzentration unter Einfluss von einem Repressor (hier die Gleichung für die Proteinproduktion reguliert durch einen Repressor einfügen)
        let Proteinkonzentration2' = beta * 1./(1.+(R/K_d)) - (gamma*Proteinkonzentration2)
        // Änderung des Proteinkonzentration unter Einfluss von einem Aktivator (hier die Gleichung für die Proteinproduktion reguliert durch einen Aktivator einfügen)
        let Proteinkonzentration3' = beta * 1./(1.+(K_d/A)) - (gamma*Proteinkonzentration3)

        [| Proteinkonzentration1'; Proteinkonzentration2';Proteinkonzentration3' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0
    0.0  
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 2 
    proteinsregulatedbyrep|> Chart.Spline 
    |> Chart.withTraceInfo("Repressor");
    let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 3 
    proteinsregulatedbyakt|> Chart.Spline 
    |> Chart.withTraceInfo("Aktivator");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.15
***Aktivator und Repressor führen beide zu einer Verringerung der maximalen Genexpression. Wenn allerdings beide zum selben Ergebnis führen, warum gibt es dann beides? Erklären sie, warum mache Gene eher von einem Aktivator reguliert werden sollten als von einem Repressor und umgekehrt.***

***Nutzen sie für die beantwortung dieser Frage Literaturquellen, falls sie die Antwort nicht wissen sollten.
Sie können selber nach passenden Literaturquellen suchen oder die nachfolgenden zwei nutzen:***<br>
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC388476/ <br>
https://www.pnas.org/doi/full/10.1073/pnas.0506610103

***Anmerkungen: Die Antworten die beide Quellen auf die Frage geben unterscheidet sich, es ist wissenschaftlich bisher noch nicht geklärt welche der Antworten die korrekte ist (es ist ausreichend wenn sie eine der beiden Antworten geben).***


Hier die Antwort eintragen.

Es folgt eine kurze Besprechung. Falls sie zu schnell vorangekommen sind, schauen sie sich bitte die zweite Quelle und Erklärungsmöglichkeit an.

### Autoregulation

#### Negative Autoregulation

Neben der Fähigkeit von Repressoren die Produktion von bestimmten Proteinen zu regulieren, sind viele auch dazu in der Lage sich selbst zu regulieren. Ein Beispiel dafür ist der Repressor LacI, der nicht nur die Expression des Lac Operons begrenzt sondern auch seine eigene. Dies wird im folgenden negative Autoregulation genannt, ein anderer Namen dafür ist auch negativer Feedbackloop (wobei dieser Ausdruck allgemeiner ist und auch für mehrere Proteine verwendet werden kann). Die Differenzialgleichung, die diesen Prozess beschreibt ist analog zu der von der Proteinkonzentration die von einem Repressor reguliert wird, nur das die Repressorkonzentration ersetzt wird mit der Proteinkonzentration, also
$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {Proteinkonzentration}{K_{D}}}  - \gamma*Proteinkonzentration$$

Aber warum sollte ein Protein seine eigene Produktion durch Autoregulation dämpfen? Wäre es nicht deutlich einfach die maximale Reproduktionsrate $\beta$ zu verringern, da eine Verringerung von $\beta$ (oder eine Zunahme von $\gamma$) zu einem geringerem $Proteinkonezntation_{GGW}$ führt (wie bereits bei Frage 2 bearbeitet)? Um diese Frage zu beantworten ist es nötig sich die Zeit anzusehen die benötigt wird um das $Proteinkonezntation_{GGW}$ zu erreichen. Das folgende Model zeigt wie sich die Proteinkonzentration, für ein Protein das ohne weitere Einflüße expremiert wird (also ohne Eigenregulation, oder Repressor/ Aktivator), verhält für verschiedene Werte von $\beta$ und $\gamma$. Beachte dabei das wenn $\beta$ varriert wird (Graph 1) gamma konstant bleibt, und wenn $\gamma$ varriert wird (Graph 2)  $\beta$ konstant bleibt.


In [ ]:
//Bitte nichts ändern!

//Konstanten
let beta1 =  0.5// Konstante für die Zunahme der Proteinkonzentration
let beta2 =  1. // Konstante für die Zunahme der Proteinkonzentration
let beta3 =  2. // Konstante für die Zunahme der Proteinkonzentration
let beta4 =  4. // Konstante für die Zunahme der Proteinkonzentration
let gamma1 = 2. // Konstante für die Verringerung der Proteinkonzentration
let gamma2 = 4. // Konstante für die Verringerung der Proteinkonzentration
let gamma3 = 8. // Konstante für die Verringerung der Proteinkonzentration
let gamma4 = 16.// Konstante für die Verringerung der Proteinkonzentration


// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->
        // Veränderung der Proteinmenge
        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge
        let Proteinkonzentration3 = P[2] // Änderung in der Proteinmenge
        let Proteinkonzentration4 = P[3] // Änderung in der Proteinmenge

        let Proteinkonzentration5 = P[4] // Änderung in der Proteinmenge
        let Proteinkonzentration6 = P[5] // Änderung in der Proteinmenge
        let Proteinkonzentration7 = P[6] // Änderung in der Proteinmenge
        let Proteinkonzentration8 = P[7] // Änderung in der Proteinmenge
        // Modelfunktion
        let Proteinkonzentration1' = beta1 - gamma2 * Proteinkonzentration1 //DGL für ein unreguliertes Protein
        let Proteinkonzentration2' = beta2 - gamma2 * Proteinkonzentration2 //DGL für ein unreguliertes Protein
        let Proteinkonzentration3' = beta3 - gamma2 * Proteinkonzentration3 //DGL für ein unreguliertes Protein
        let Proteinkonzentration4' = beta4 - gamma2 * Proteinkonzentration4 //DGL für ein unreguliertes Protein
 
        let Proteinkonzentration5' = beta3 - gamma1 * Proteinkonzentration5 //DGL für ein unreguliertes Protein
        let Proteinkonzentration6' = beta3 - gamma2 * Proteinkonzentration6 //DGL für ein unreguliertes Protein
        let Proteinkonzentration7' = beta3 - gamma3 * Proteinkonzentration7 //DGL für ein unreguliertes Protein
        let Proteinkonzentration8' = beta3 - gamma4 * Proteinkonzentration8 //DGL für ein unreguliertes Protein

        [|Proteinkonzentration1';Proteinkonzentration2';Proteinkonzentration3';Proteinkonzentration4';Proteinkonzentration5';Proteinkonzentration6';Proteinkonzentration7';Proteinkonzentration8' |]

//Startkonzentrationen
let P0 = [| 
    0.0 
    0.0 
    0.0 
    0.0 
    0.0
    0.0
    0.0
    0.0
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 15 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

// Generierung der Punkte für das Kennzeichnen der Antwortszeit
let t0 = log(2.) / gamma2
let x1 = beta1 / gamma2 * (0.5)
let x2 = beta2 / gamma2 * (0.5)
let x3 = beta3 / gamma2 * (0.5)
let x4 = beta4 / gamma2 * (0.5)
let t1 = log(2.) / gamma1
let t2 = log(2.) / gamma2
let t3 = log(2.) / gamma3
let t4 = log(2.) / gamma4
let x5 = beta3 / gamma1 * (0.5)
let x6 = beta3 / gamma2 * (0.5)
let x7 = beta3 / gamma3 * (0.5)
let x8 = beta3 / gamma4 * (0.5)

//Graphische Darstellung der numerischen Lösung
[
    [
        let proteins = Ps |> SolPoints.toPoints 1 
        proteins |> Chart.Spline 
        |> Chart.withTraceInfo("beta = 0.5");
        let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 2 
        proteinsregulatedbyrep|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 1.0");
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 3 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 2.0");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword ForestGreen)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 4 
        proteinsregulatedbyakt|> Chart.Spline 
        |> Chart.withTraceInfo("beta = 4.0");
        [t0,x1]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("beta = 0.5");
        [t0,x2]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("beta = 1.0");
        [t0,x3]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Green)
        |> Chart.withTraceInfo("beta = 2.0");
        [t0,x4]
        |> Chart.Point 
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Purple)
        |> Chart.withTraceInfo("beta = 4.0");
    ]
    |> Chart.combine
    |> Chart.withTitle @"Veränderung von beta"
    |> Chart.withXAxisStyle("Zeit")
    |> Chart.withYAxisStyle("Proteinkonzentration")

    [
        let proteins = Ps |> SolPoints.toPoints 5 
        proteins 
        |> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 2.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword DarkViolet)
        let proteinsregulatedbyrep = Ps |> SolPoints.toPoints 6 
        proteinsregulatedbyrep
        |> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 4.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword ForestGreen)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 7 
        proteinsregulatedbyakt
        |> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 8.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        let proteinsregulatedbyakt = Ps |> SolPoints.toPoints 8 
        proteinsregulatedbyakt
        |> Chart.Spline 
        |> Chart.withTraceInfo("gamma = 16.");
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        [t1,x5]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword DarkViolet)
        |> Chart.withTraceInfo("gamma = 2.");
        [t2,x6]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Green)
        |> Chart.withTraceInfo("gamma = 4.0");
        [t3,x7]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("gamma = 8.0");
        [t4,x8]
        |> Chart.Point 
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("gamma = 16.0");
    ]
    |> Chart.combine
    |> Chart.withTitle @"Veränderung von beta                                       Veränderung von gamma"
    |> Chart.withXAxisStyle("Zeit")
    |> Chart.withYAxisStyle("Proteinkonzentration")

]
|> Chart.Grid (1,2)
|> Chart.withSize(1000,500)

<!-- Plotly chart will be drawn inside this DIV -->

Die Graphen zeigen, dass sowohl die Abnahme von $\beta$, als auch die Zunahme von $\gamma$ zu einer Verringerung der $Proteinkonzentration_{GGW}$ führen. Allerdings unterscheidet sich die Form der Graphen erheblich. 
Acht Punkte sind makiert, die diesen Fakt unterstreichen. Diese repräsentieren die Antwortszeit, welche beschreibt wie lange es dauert, bis der Wert der $Proteinkonzentration (t)$ gleich dem Wert $\frac {1}{2}*Proteinkonzentration_{GGW}$ ist. 
Die Betrachtung der Antwortszeit zeigt, dass eine Veränderung von $\beta$ nicht zu einer Verkürzung der Antwortszeit führt, während eine Veränderung von $\gamma$ zu einer Verringerung der Antwortszeit führt. Die Formel für die Berechnung der Antwortszeit ist
$$ T_{\frac {1}{2}} = \frac  {\log (2)}{\gamma}$$
wobei $T_{\frac {1}{2}}$ die Antwortszeit repräsentiert. Dies bedeutet, wenn ein Protein möglichst schnell produziert werden soll, sollte der Wert für $\gamma$ möglichst groß sein (Anmerkung: Die gezeigte Formel gilt nur für unregulierte Proteine).

##### Aufgabe 3.16

***Überlege was ein hoher Wert für $\gamma$ im Bezug auf die Proteinstabilität bedeutet (zur Erinnerung $\gamma$ beschreibt sowohl den aktiven Abbau, als auch die Verteilung in der Zelle beim Wachstum).
Tipp: Falls sie Probleme mit dieser Aufgabe haben, überlegen sie sich welche Faktoren einen schnellen beziehungsweise langsamen Abbau von einem Protein begünstigen.***



Bitte hier die Antwort eingeben.

##### Aufgabe 3.17

***Betrachten sie nun den Graphen für ein Protein, das sich selbst negativ autoreguliert.
Setzen sie die Gleichung für negative Autoregulation in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Proteinkonzentration als "Proteinkonzentration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***

In [ ]:
//Konstanten
let beta  =  2.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0 // Konstante für die Verringerung der Proteinkonzentration
let K_d =    1.0 // Dissoziationskonstante

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge (unreguliert)
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge (reguliert durch negative Autoregulation)
        //Änderung der Proteinkonzentration ohne negative Autoregulation
        let Proteinkonzentration1' = beta - (gamma*Proteinkonzentration1)
        // Änderung des Proteinkonzentration bei negativer Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonzentration2' = beta * 1./(1.+(Proteinkonzentration2/K_d)) - (gamma*Proteinkonzentration2)

        [| Proteinkonzentration1'; Proteinkonzentration2' |]

//Startkonzentrationen
let P0 = [| 
    0.0
    0.0  
|]

// Generierung der Punkte für das Kennzeichnen der Antwortszeit
let t0 = log(2.) / gamma
let t1 = K_d/(2. * beta) + 0.135
let x0 = beta / gamma * (0.5) 
let x1 = K_d * 0.5 

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 12 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("negative AR");
    [t0,x0]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Blue)
        |> Chart.withTraceInfo("Unreguliert");
    [t1,x1]
        |> Chart.Point
        |> Chart.withMarkerStyle(Color = Color.fromKeyword Red)
        |> Chart.withTraceInfo("negative AR");    
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.18

***Welchen Effekt hat die negative Autoregulation im Bezug auf die Antwortzeit.*** 


Bitte hier die Antwort eingeben.


##### Aufgabe 3.19

***Warum ist negative Autoregulation ein sinnvollerer Mechanismus für eine verkürzung der Antwortszeit für die meisten Proteine im Vergleich zur Erhöhung von $\gamma$?***


Bitte hier die Antwort eingeben.

#### Positive Autoregulation

Die Betrachtung der negativen Autoregulation zeigte, dass sie dazu genutzt werden kann, die Antwortgeschwindigkeit der Proteinkonzentration zu erhöhen. Beim Vergleich von Repressor und Aktivator hatten beide einen identischen Effekt. Beide verringerten die $Proteinkonzentration_{GGW}$ und hatten keinen Einfluss auf die Antwortgeschwindigkeit. Daher ist die logische Folgerung, dass sich postive Autoregulation genauso wie die negative Autoregulation verhält. Diese Vermutung soll im nachfolgendem Abschnitt untersucht werden. Eine durch positive Autoregulation regulierte Genexpression wird durch folgende Differenzialgleichung dargestellt:

$$ \frac {dProteinkonzentration} {dt} = \beta* \frac {1}{1+ \frac {K_{D}}{Proteinkonzentration}}  - \gamma*Proteinkonzentration$$



##### Aufgabe 3.20

***Setzen sie die Gleichung für postive Autoregulation in die nachfolgende Simulation ein (Bitte beachten sie, dass sie die Proteinkonzentration als "Proteinkonzentration2" einsetzten müssen, ohne "". Beachten sie ebenfalls das $\beta \textrm{ und } \gamma$ als "beta" und "gamma", ohne "", eingesetzt werden müssen und nicht als Symbole, und das $K_{D}$ als K_d eingesetzt wird).***

In [ ]:
//Konstanten
let beta  =  10.0 // Konstante für die Zunahme der Proteinkonzentration
let gamma =  1.0  // Konstante für die Verringerung der Proteinkonzentration
let K_d =    4.0  // Dissoziationskonstante
let n =      2.0  // Hill Koeffizent 

// unsere DGL als  Modell (Model)
let dP_dt : Model = 
    fun P t ->

        let Proteinkonzentration1 = P[0] // Änderung in der Proteinmenge
        let Proteinkonzentration2 = P[1] // Änderung in der Proteinmenge
        //Änderung der Proteinkonzentration ohne positive Autoregulation
        let Proteinkonzentration1' = beta - (gamma*Proteinkonzentration1)
        // Änderung des Proteinkonzentration bei positiver Autoregulation (hier die Gleichung einsetzen)
        let Proteinkonzentration2' = beta * (Proteinkonzentration2**n)/((Proteinkonzentration2**n) + (K_d**n)) - (gamma*Proteinkonzentration2)

        [| Proteinkonzentration1'; Proteinkonzentration2' |]

//Startkonzentrationen
let P0 = [| 
    //Startkonzentration für das unregulierte Protein (muss nicht verändert werden)
    0.0 
    //Startkonzentration für das durch postive Autoreguliation regulierte Protein (hier bitte die Startwerte ändern)
    4.0 
|]

// Simulation der Modelfuntkion
let Ps = 
    modelContext.OdeInt(
        0., // Startzeitpunkt
        P0,
        dP_dt
    )
    |> SolPoints.take 10 // Anzahl der Zeitpunkte
    |> SolPoints.memorize 

//Graphische Darstellung der numerischen Lösung
[
    let proteins = Ps |> SolPoints.toPoints 1 
    proteins |> Chart.Spline 
    |> Chart.withTraceInfo("Unreguliert");
    let proteinsregulated = Ps |> SolPoints.toPoints 2 
    proteinsregulated|> Chart.Spline 
    |> Chart.withTraceInfo("positive AR");
]

|> Chart.combine
|> Chart.withXAxisStyle("Zeit")
|> Chart.withYAxisStyle("Proteinkonzentration")

<!-- Plotly chart will be drawn inside this DIV -->

##### Aufgabe 3.21

***Welchen Effekt hat positive Autoregulation?***

Hier die Antwort eingeben.

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen.


##### Expertenaufgabe 3.2

***Überlegen sie sich wie lange die Antwortszeit von einem Protein ist, mit folgenden Bedingungen:***
1. ***Es gibt eine gewisse Anfangskonzentration $P_{Start}$***
2. ***Die Produktionsrate $\beta$ ist null***
3. ***Der Teil $\gamma_{Abbau}$ von $\gamma$ ist null.*** 

***Das bedeutet die initale Proteinkonzentration verändert sich nur durch das Verteilen des Proteins in der Zelle und durch Zellteilung. Die Antwortszeit soll sich auf das Volumen von einer Zelle beziehen.
Überlegen sie weiterhin, ob diese Bedingungen häufig auftreten insbesondere mit Blick auf die Stabilität der meisten Proteine in der Zelle.***

***Falls sie Probleme mit dem lösen der Aufgabe haben, nutzen sie die folgende Literatur (diese beschreibt ein etwas komplexeres System, kommt aber zu der gesuchten Antwort):
https://www.sciencedirect.com/science/article/pii/S0022283603005060?via%3Dihub***

Bitte hier die Antwort eingeben.

Take Home Messages

1. Ordinäre differentialgleichungen für die Änderung der Proteinkonzentration erlauben es einfache Genexpressions prozesse zu beschreiben
2. 
..
n. <br>
n+1. Die Regulation der Proteinproduktion durch Aktivatoren und Repressoren führt zu einer Verringerung des Proteinkonzentrationsgleichgewichtspunktes.<br>
n+2. Negative Autoregulation verringert die Antwortszeit der Proteinproduktion und ist eines der einfachsten Netzwerkmotive.<br>
n+2. Postive Autoregulation führt zu einer erhöhten Antwortszeit und ist ebenfalls ein Netzwerkmotiv.<br>
